In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data= pd.read_csv('../input/trip-advisor-hotel-reviews/tripadvisor_hotel_reviews.csv')

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_curve,auc
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, train_test_split
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import seaborn as sns
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
data.isnull().sum()
di={1:"0",2:"1",3:"2",4:"3",5:"4"}
y=data["Rating"].map(di)
stop_words=stopwords.words('english')

In [ ]:
def remove_stopwords(txt):
    words=word_tokenize(txt)
    words=[w for w in words if w.isalpha()]
    words=[w for w in words if not w in stop_words]
    return ' '.join(words)

data.Review = data["Review"].apply(remove_stopwords)

sia=SentimentIntensityAnalyzer()

dict_sent=data["Review"].apply(lambda z:sia.polarity_scores((z)))

positive=[]
negative=[]
neutral=[]
for w in dict_sent:
    positive.append(w['pos'])
    negative.append(w['neg'])
    neutral.append(w['neu'])

dic={"positive":positive,"negative":negative,"neutral":neutral}
data2=pd.DataFrame(dic)
data2["Rating"]=y

In [ ]:
sns.boxplot(x="Rating",y="positive",data=data2)

In [ ]:
sns.boxplot(x="Rating",y="negative",data=data2)

In [ ]:
sns.boxplot(x="Rating",y="neutral",data=data2)

In [ ]:
def remove_outliers(data):
    Q1=np.quantile(data,0.25)
    Q3=np.quantile(data,0.75)
    IQR=Q3-Q1
    low=Q1-1.5*IQR
    up=Q3+1.5*IQR
    return up, low


up,low=remove_outliers(data2["neutral"])

x=data2[(data2["neutral"]>low)&(data2["neutral"]<up)]
x.isna().sum()
y=x["Rating"]
x.drop("Rating",axis=1,inplace=True)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

LR=LogisticRegression(solver="lbfgs",class_weight="balanced",max_iter=10000)
model_LR=LR.fit(x_train,y_train)
KNN=KNeighborsClassifier()
model_KNN=KNN.fit(x_train,y_train)
DTC=DecisionTreeClassifier(random_state=0)
model_DTC=DTC.fit(x_train,y_train)
RFC=RandomForestClassifier(random_state=0)
model_RFC=RFC.fit(x_train,y_train)
GBC=GradientBoostingClassifier(random_state=0)
model_GBC=GBC.fit(x_train,y_train)

In [ ]:
models=[model_LR,model_KNN,model_DTC,model_RFC,model_GBC]
for model in models:
    name=model.__class__.__name__
    R2=cross_val_score(model,x_test,y_test,cv=10,verbose=False).mean()
    error=-cross_val_score(model,x_test,y_test,cv=10,scoring="neg_mean_squared_error",verbose=False).mean()
    print(name + ":")
    print("*"*20)
    print(R2)
    print(np.sqrt(error))
    print("*"*20)

In [ ]:
prediction=GBC.predict(x_test)
print(classification_report(y_test,prediction))